### 本篇代码用于搜集目标股票的金融数据——历史股价，市场数据等。这些数据根据每一个8-k文件的发布来进行计算，形成feature

In [1]:
import pandas as pd
import requests
import numpy as np
from config import Config
import pandas_market_calendars as mcal
import datetime
from tqdm import tqdm
import io
import dateutil.relativedelta
from time import sleep
import gc
from pandas.core.frame import DataFrame
pd.options.mode.chained_assignment = None #取消赋值警告

#### 搜集历史股价数据

In [2]:
top100 = pd.read_csv('top100.csv')#导入前100股票的cik

In [3]:
top100.head()

,ticker
0,MSFT
1,AAPL
2,AMZN
3,FB
4,BRK.B


In [76]:

all_tickers = dict()
#从alphavantage得到api
av_key = "CB9VT9UNDC5VBKYU"

base_url = "https://www.alphavantage.co/query?"
params = {"function":"TIME_SERIES_DAILY_ADJUSTED",
          "datatype":"csv",
          "outputsize":"full",
          "apikey": av_key}

for ticker in tqdm(top100['ticker']):
    params["symbol"] = ticker
    r = requests.get(base_url,params)
    filepath = io.StringIO(r.content.decode('utf-8'))
    av_df = pd.read_csv(filepath,parse_dates=True,index_col=[0],error_bad_lines=False)
    all_tickers[ticker] = av_df
    sleep(1)



#df1.to_pickle("Pickles\\missing_data2.pkl")
#missing_data2.to_csv("missing_data2.csv")
all_tickers_data = pd.concat(all_tickers,sort=False)
all_tickers_data.to_pickle("Pickles\\all_tickers_data.pkl")
all_tickers_data.to_csv("all_tickers_data.csv",chunksize=50)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.55s/it]


In [80]:
all_tickers_data.head()

adjusted_close   close  dividend_amount      high     low  \
     timestamp                                                               
AAPL 2019-07-22          207.22  207.22              0.0  207.2300  203.61   
     2019-07-19          202.59  202.59              0.0  206.5000  202.36   
     2019-07-18          205.66  205.66              0.0  205.8800  203.70   
     2019-07-17          203.35  203.35              0.0  205.0915  203.27   
     2019-07-16          204.50  204.50              0.0  206.1100  203.50   

                   open  split_coefficient    volume  
     timestamp                                        
AAPL 2019-07-22  203.65                1.0  22207558  
     2019-07-19  205.79                1.0  20929307  
     2019-07-18  204.00                1.0  18582161  
     2019-07-17  204.05                1.0  14107450  
     2019-07-16  204.59                1.0  16866816

#### 搜集指数数据——gspc&vix (网上下载)

In [8]:
gspc = pd.read_csv('Data\Indexes\gspc.csv')
vix = pd.read_csv('Data\\Indexes\\vix.csv')

In [4]:
len(gspc)

1278

In [9]:
gspc.head()

,Date,Adj Close,Open,High,Low
0,"Jul 19, 2019",2976.61,3004.26,3006.02,2975.86
1,"Jul 18, 2019",2995.11,2978.87,2998.28,2973.09
2,"Jul 17, 2019",2984.42,3005.10,3005.26,2984.25
3,"Jul 16, 2019",3004.04,3012.13,3015.02,3001.15
4,"Jul 15, 2019",3014.30,3017.80,3017.80,3008.77


In [11]:
def str_to_time(time):#写个函数将string的时间变成时间格式
    return datetime.datetime.strptime(time,"%b %d, %Y")

In [12]:
gspc['Date'] = gspc['Date'].apply(str_to_time)

In [13]:
gspc.head()

,Date,Adj Close,Open,High,Low
0,2019-07-19,2976.61,3004.26,3006.02,2975.86
1,2019-07-18,2995.11,2978.87,2998.28,2973.09
2,2019-07-17,2984.42,3005.10,3005.26,2984.25
3,2019-07-16,3004.04,3012.13,3015.02,3001.15
4,2019-07-15,3014.30,3017.80,3017.80,3008.77


In [14]:
len(vix)

1280

In [15]:
vix.head()

,Date,Adj Close,Open,High,Low
0,"Jul 22, 2019",14.44,14.54,14.70,14.34
1,"Jul 19, 2019",14.45,13.31,14.45,13.09
2,"Jul 18, 2019",13.53,14.45,14.50,13.19
3,"Jul 17, 2019",13.97,12.62,13.97,12.24
4,"Jul 16, 2019",12.86,12.61,13.14,12.28


In [16]:
vix['Date'] = vix['Date'].apply(str_to_time)

In [17]:
vix.head()

,Date,Adj Close,Open,High,Low
0,2019-07-22,14.44,14.54,14.70,14.34
1,2019-07-19,14.45,13.31,14.45,13.09
2,2019-07-18,13.53,14.45,14.50,13.19
3,2019-07-17,13.97,12.62,13.97,12.24
4,2019-07-16,12.86,12.61,13.14,12.28


In [32]:
gspc.to_pickle('Pickles\\gspc.pkl')
vix.to_pickle('Pickles\\vix.pkl')

#### 根据8-K文件计算股价波动，并将金融数据和文本数据整合

交易日函数：检查涉及的日期是否是交易日

In [86]:
    #如果日期是非交易日，则往前推到最近的交易日
    nyse = mcal.get_calendar('NYSE')
    nyse_holidays = nyse.holidays().holidays
    def weekday_check_before(date):  
        # isoweekday里周一是1周天是7
        while date.isoweekday() > 5 or date.date() in nyse_holidays:
            date = date + datetime.timedelta(days=-1)#不断前推1天直到跳出循环
        return date
    #如果日期是非交易日，则往后推到最近的交易日
    def weekday_check_after(date):  
        # isoweekday里周一是1周天是7
        while date.isoweekday() > 5 or date.date() in nyse_holidays:
            date = date + datetime.timedelta(days=1)#不断后推1天直到跳出循环
        return date

价差函数：计算两个价格之间的百分比变化，该函数将用在后面的函数中执行计算功能

In [87]:
    #计算价格的百分比变化
    def calculate_pct_change(end_value,start_value):
        pct_change = (end_value - start_value) / start_value
        pct_change = round(pct_change,4) * 100
        return pct_change

平均股指函数：计算两个日期之间的市场指数平均值，该函数将用在后面的函数中执行计算功能

In [88]:
    # 与平均股价计算类似，这个函数计算两个时点的平均股指
    gspc_df = pd.read_pickle("Pickles\\gspc.pkl")
    gspc_df = gspc_df.set_index('Date')
    def get_index_price(start_date,end_date,market_open=False):
        try:
            if market_open == True:#datetime64是numpy的y/m/r h/m/s数据
                price = gspc_df.loc[(gspc_df.index >= np.datetime64(start_date.date())) & 
                                 (gspc_df.index <= np.datetime64(end_date)),"Open"].mean()
            else:
                price = gspc_df.loc[(gspc_df.index >= np.datetime64(start_date.date())) & 
                                 (gspc_df.index <= np.datetime64(end_date)),"Adj Close"].mean()
        except IndexError:
                price = np.nan
        return price

In [89]:
all_tickers_data = pd.read_pickle("Pickles\\all_tickers_data.pkl")

平均股价函数：计算两个时点之间的股价平均值，该函数将用在后面的函数中执行计算功能

In [90]:
    #这个函数的目的就是计算指定两个时间之间的平均股票价格，注意不是信息发布前后
    #比如信息发布前一周的平均价格。如果start date = end date相当于取当天的价格
    #可以是平均开盘价也可以是平均收盘价

    def get_av_data(ticker,start_date,end_date,market_open=False):
        start_date = start_date.date()
        end_date = end_date.date()

        try:
            if market_open == False:#如果是false就计算调整后的平均收盘价
                #.xs是cross section切片的意思
                price = all_tickers_data.xs(ticker,0).loc[end_date:start_date,"adjusted_close"].mean()
            else:#如果是true就计算平均开盘价
                price = all_tickers_data.xs(ticker,0).loc[end_date:start_date,"open"].mean()
        except (KeyError,IndexError):
            price = np.nan
        return price

历史表现函数：得到某一8K文件发布时，该股票过去一周，一月，一季度，和一年的历史股价表现

In [91]:
#Takes datetime object and ticker string, returns price (opening or closing)
    def get_historical_movements(row,period):
        ticker,release_date = row[0],row[1]

        #这里用到了datetime.timedelta和dateutil.relativedelta，都是计算duration，后者是前者标准库的延伸
        #relativedelta功能更多，可以计算比较复杂的时间差，但timedelta更快，虽然只能计算天
       #1 Week
        if period == "week": #计算一周价格变动，取这周头尾1天
            e_start = release_date + datetime.timedelta(weeks=-1)#消息发布时间前1周的时间
            b_start = e_start

            e_end = release_date + dateutil.relativedelta.relativedelta(days=-1)#前一天的时间
            b_end = e_end

         #1 Month    
        elif period == "month":# 计算一个月价格变动，取头尾各5天。某一天难以代表那段时间的价格。
            e_start = release_date + dateutil.relativedelta.relativedelta(months=-1)#前一个月
            b_start = e_start + dateutil.relativedelta.relativedelta(days=-5)

            e_end = release_date + dateutil.relativedelta.relativedelta(days=-1)
            b_end = release_date + dateutil.relativedelta.relativedelta(days=-6)

        #1 Quarter
        elif period == "quarter":# 一季度，取头尾10天
            e_start = release_date + dateutil.relativedelta.relativedelta(months=-3)
            b_start = e_start + dateutil.relativedelta.relativedelta(days=-10)

            e_end = release_date + dateutil.relativedelta.relativedelta(days=-1)
            b_end = release_date + dateutil.relativedelta.relativedelta(days=-11)

        #1 Year
        elif period == "year":#一年，取头尾20天
            e_start = release_date + dateutil.relativedelta.relativedelta(years=-1)
            b_start = e_start + dateutil.relativedelta.relativedelta(days=-20)

            e_end = release_date + dateutil.relativedelta.relativedelta(days=-1)
            b_end = release_date + dateutil.relativedelta.relativedelta(days=-21)
        else:
            raise KeyError

        #将非交易日推移至交易日
        e_start = weekday_check_before(e_start)
        b_start = weekday_check_before(b_start)
        e_end = weekday_check_after(e_end)
        b_end = weekday_check_after(b_end)

        #计算历史股价变动百分比，过去一年，一季度，一个月，一周
        #以1年为例，1年前的20天平均价格，和刚过去的20天平均价格，做差
        start_price = get_av_data(ticker=ticker,start_date = b_start, end_date = e_start)
        end_price = get_av_data(ticker=ticker,start_date = b_end, end_date = e_end)
        stock_change = calculate_pct_change(end_price,start_price)

        #计算指数，同上
        start_index = get_index_price(start_date = b_start, end_date = e_start)
        end_index = get_index_price(start_date = e_start, end_date = e_end)
        index_change =  calculate_pct_change(end_index,start_index)

        normalized = stock_change - index_change
        return normalized

In [92]:
vix_df = pd.read_pickle("Pickles\\vix.pkl")

In [93]:
vix_df = vix_df.set_index('Date')

股价变化函数：计算某一文件发布引起的股价变动百分比

In [151]:
    # Takes ticker, 8K release date, checks time of release and then calculate before and after price change
    def get_change_price(row):
        release_date = row['release_date']
        ticker = row['ticker']
        #得到信息发布当天的开盘收盘时间
        market_close = release_date.replace(hour=16,minute=0,second=0)
        market_open = release_date.replace(hour=9,minute=30,second=0)

        #如果发布时间是当天收盘之后，那么股价变化是当天的收盘价与第二个交易日的开盘价之差
    # If report is released after market hours, take change of start date close and release date open
        if release_date > market_close:
            start_date = release_date
            end_date = release_date + datetime.timedelta(days=1)
            start_date = weekday_check_before(start_date)
            end_date = weekday_check_after(end_date)

            price_before_release = get_av_data(ticker,start_date,start_date,market_open=False)
            price_after_release = get_av_data(ticker,end_date,end_date,market_open=True)

            index_before_release = get_index_price(start_date,start_date,market_open=False)
            index_after_release = get_index_price(end_date,end_date,market_open=True)

            try: #当天收盘时的VIX（这个指数只是在模型中充当控制变量）
                vix = vix_df.loc[vix_df.index == np.datetime64(start_date.date()),"Adj Close"][0].item()
            except IndexError:
                vix = np.nan #nan - not a number

            #如果发布时间是当天开盘之前，那么股价变化是前一交易日的收盘价与当天的开盘价之差
        # If report is released before market hours, take change of start date's close and release date's open
        elif release_date < market_open:
            start_date = release_date + datetime.timedelta(days=-1)
            end_date = release_date
            start_date = weekday_check_before(start_date)
            end_date = weekday_check_after(end_date)

            price_before_release =get_av_data(ticker,start_date,start_date,market_open=False)
            price_after_release = get_av_data(ticker,end_date,end_date,market_open=True) 

            index_before_release = get_index_price(start_date,start_date,market_open=False)
            index_after_release = get_index_price(end_date,end_date,market_open=True)
            try: #前一天收盘VIX
                vix = vix_df.loc[vix_df.index == np.datetime64(start_date.date()),"Adj Close"][0].item()
            except IndexError:
                vix = np.nan
        #如果发布时间是交易时段，那么股价变化就是当天开盘收盘价差
        # If report is released during market hours, use market close
        else:
            start_date = release_date
            end_date = release_date
            start_date = weekday_check_before(start_date)
            end_date = weekday_check_after(end_date)
            price_before_release = get_av_data(ticker,start_date,start_date,market_open=True)
            price_after_release = get_av_data(ticker,end_date,end_date,market_open=False)

            index_before_release = get_index_price(start_date,start_date,market_open=True)
            index_after_release = get_index_price(end_date,end_date,market_open=False)
            
            try: #当天的开盘VIX
                vix = vix_df.loc[vix_df.index == np.datetime64(start_date.date()),"Open"][0].item()
            except IndexError:
                vix = np.nan
         #计算信息发布前后价格和指数的百分比变化       
        price_pct_change = calculate_pct_change(price_after_release,price_before_release)
        index_pct_change = calculate_pct_change(index_after_release,index_before_release)
        #去掉市场波动
        normalized_change = price_pct_change - index_pct_change

        return normalized_change

股指变化函数：计算某一文件发布前后市场的变化

In [149]:
    # Takes ticker, 8K release date, checks time of release and then calculate before and after price change
    def get_change_vix(row):
        release_date = row['release_date']
        ticker = row['ticker']
        #得到信息发布当天的开盘收盘时间
        market_close = release_date.replace(hour=16,minute=0,second=0)
        market_open = release_date.replace(hour=9,minute=30,second=0)

        #如果发布时间是当天收盘之后，那么股价变化是当天的收盘价与第二个交易日的开盘价之差
    # If report is released after market hours, take change of start date close and release date open
        if release_date > market_close:
            start_date = release_date
            end_date = release_date + datetime.timedelta(days=1)
            start_date = weekday_check_before(start_date)
            end_date = weekday_check_after(end_date)

            #price_before_release = get_av_data(ticker,start_date,start_date,market_open=False)
            #price_after_release = get_av_data(ticker,end_date,end_date,market_open=True)

            index_before_release = get_index_price(start_date,start_date,market_open=False)
            index_after_release = get_index_price(end_date,end_date,market_open=True)

            try: #当天收盘时的VIX（这个指数只是在模型中充当控制变量）
                vix = vix_df.loc[vix_df.index == np.datetime64(start_date.date()),"Adj Close"][0].item()
            except IndexError:
                vix = np.nan #nan - not a number

            #如果发布时间是当天开盘之前，那么股价变化是前一交易日的收盘价与当天的开盘价之差
        # If report is released before market hours, take change of start date's close and release date's open
        elif release_date < market_open:
            start_date = release_date + datetime.timedelta(days=-1)
            end_date = release_date
            start_date = weekday_check_before(start_date)
            end_date = weekday_check_after(end_date)

            #price_before_release =get_av_data(ticker,start_date,start_date,market_open=False)
            #price_after_release = get_av_data(ticker,end_date,end_date,market_open=True) 

            index_before_release = get_index_price(start_date,start_date,market_open=False)
            index_after_release = get_index_price(end_date,end_date,market_open=True)
            try: #前一天收盘VIX
                vix = vix_df.loc[vix_df.index == np.datetime64(start_date.date()),"Adj Close"][0].item()
            except IndexError:
                vix = np.nan
        #如果发布时间是交易时段，那么股价变化就是当天开盘收盘价差
        # If report is released during market hours, use market close
        else:
            start_date = release_date
            end_date = release_date
            start_date = weekday_check_before(start_date)
            end_date = weekday_check_after(end_date)
            #price_before_release = get_av_data(ticker,start_date,start_date,market_open=True)
            #price_after_release = get_av_data(ticker,end_date,end_date,market_open=False)

            index_before_release = get_index_price(start_date,start_date,market_open=True)
            index_after_release = get_index_price(end_date,end_date,market_open=False)
            
            try: #当天的开盘VIX
                vix = vix_df.loc[vix_df.index == np.datetime64(start_date.date()),"Open"][0].item()
            except IndexError:
                vix = np.nan
         #计算信息发布前后价格和指数的百分比变化       
        #price_pct_change = calculate_pct_change(price_after_release,price_before_release)
        #index_pct_change = calculate_pct_change(index_after_release,index_before_release)
        #去掉市场波动
        #normalized_change = price_pct_change - index_pct_change

        return vix

#### 将前面定义的函数用来依次作用在搜集的金融数据上，得到相应的feature

In [95]:
df = pd.read_pickle('Pickles\\sec_df.pkl')

In [96]:
df.head()

,GICS Sector,GICS Sub Industry,cik,doc_name,items,release_date,text,ticker,txt_link
0,Information Technology,Information Technology,789019.0,0001193125-19-022553.txt,"[Item 2.02, Item 9.01]",2019-01-30 16:03:36,0001193125-19-022553.txt : 20190130 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...
1,Information Technology,Information Technology,789019.0,0001193125-18-337951.txt,[Item 5.07],2018-11-29 15:29:57,0001193125-18-337951.txt : 20181129 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...
2,Information Technology,Information Technology,789019.0,0001193125-18-306365.txt,"[Item 2.02, Item 9.01]",2018-10-24 16:03:06,0001193125-18-306365.txt : 20181024 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...
3,Information Technology,Information Technology,789019.0,0001193125-18-277602.txt,"[Item 5.02, Item 9.01]",2018-09-19 16:15:58,0001193125-18-277602.txt : 20180919 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...
4,Information Technology,Information Technology,789019.0,0001193125-18-221458.txt,"[Item 2.02, Item 9.01]",2018-07-19 16:02:35,0001193125-18-221458.txt : 20180719 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...


依次计算需要的历史价格变量

In [153]:

#df['rm_week'] = df[['ticker','release_date']].apply(get_historical_movements,period="week",axis=1)
#df['rm_month'] = df[['ticker','release_date']].apply(get_historical_movements,period="month",axis=1)
#df['rm_qtr'] = df[['ticker','release_date']].apply(get_historical_movements,period="quarter",axis=1)
#df['rm_year'] = df[['ticker','release_date']].apply(get_historical_movements,period="year",axis=1)
#df['vix'] = df[['ticker','release_date']].apply(get_change_vix,axis=1)
#df['price_change'] = df[['ticker','release_date']].apply(get_change_price,axis=1)
df["signal"] = df['price_change'].map(lambda x: "stay" if -1<x<1 else ("up" if x>1 else "down"))
df.head()

,GICS Sector,GICS Sub Industry,cik,doc_name,items,release_date,text,ticker,txt_link,rm_week,rm_month,rm_qtr,rm_year,vix,price_change,signal
0,Information Technology,Information Technology,789019.0,0001193125-19-022553.txt,"[Item 2.02, Item 9.01]",2019-01-30 16:03:36,0001193125-19-022553.txt : 20190130 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,-3.80,0.61,2.12,20.47,17.66,-1.82,down
1,Information Technology,Information Technology,789019.0,0001193125-18-337951.txt,[Item 5.07],2018-11-29 15:29:57,0001193125-18-337951.txt : 20181129 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,6.77,0.11,-0.01,23.76,18.65,-0.95,stay
2,Information Technology,Information Technology,789019.0,0001193125-18-306365.txt,"[Item 2.02, Item 9.01]",2018-10-24 16:03:06,0001193125-18-306365.txt : 20181024 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,-0.91,-1.60,1.60,37.85,25.23,4.71,up
3,Information Technology,Information Technology,789019.0,0001193125-18-277602.txt,"[Item 5.02, Item 9.01]",2018-09-19 16:15:58,0001193125-18-277602.txt : 20180919 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,1.02,2.87,9.60,42.83,11.75,1.35,up
4,Information Technology,Information Technology,789019.0,0001193125-18-221458.txt,"[Item 2.02, Item 9.01]",2018-07-19 16:02:35,0001193125-18-221458.txt : 20180719 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,0.66,5.09,8.96,38.96,12.87,5.21,up


In [156]:
df.to_pickle("Pickles\\text_and_finance.pkl")